In [0]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt

In [0]:
#Nomes para as colunas
columns = ["Estacao","Data","Hora","Prec","TempMax","TempMin","Insolacao","EvapoPiche","TempCompMedia",
           "UmidRelatMedia", "VelocVento", "Missing"]

#Tipos de dados em cada coluna
types = {"Hora":np.uint16,"Prec":np.float64,"TempMax":np.float64,"TempMin":np.float64,"Insolacao":np.float64,
         "EvapoPiche":np.float64,"TempCompMedia":np.float64,"UmidRelatMedia":np.float64,"VelocVento":np.float64}

#Leitura do arquivo csv
dataSet = pd.read_csv("https://github.com/gilvandrocesardemedeiros/ClimatePrediction/raw/master/Dados_INMET/Diarios/INMET-Dados_Diarios_Natal_1968-2018.csv",
                      sep=';',skiprows=48, nrows = 27003, usecols = columns[1:11], names=columns, dtype = types)

In [0]:
#Transformando data em variável do tipo datetime
dataSet["Data"] = pd.to_datetime(dataSet["Data"], format = "%d/%m/%Y")

In [4]:
#Verificando dados
dataSet.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
0,1968-01-01,0,NaN,30.6,NaN,11.1,4.4,25.14,73.50,2.666667
1,1968-01-01,1200,0.0,NaN,21.4,NaN,NaN,NaN,NaN,NaN
2,1968-01-02,0,NaN,31.0,NaN,11.9,6.5,24.92,74.75,1.666667
3,1968-01-02,1200,0.0,NaN,19.6,NaN,NaN,NaN,NaN,NaN
4,1968-01-03,0,NaN,31.6,NaN,11.3,5.0,24.90,77.25,2.333333


In [5]:
#Verificando o tipo dos dados
dataSet.dtypes

Data              datetime64[ns]
Hora                      uint16
Prec                     float64
TempMax                  float64
TempMin                  float64
Insolacao                float64
EvapoPiche               float64
TempCompMedia            float64
UmidRelatMedia           float64
VelocVento               float64
dtype: object

In [0]:
#Separando os dados em dois dataframes, um para as 00:00 h e outro para as 12:00 h
dataSet00, dataSet12 = dataSet[dataSet["Hora"] == 0], dataSet[dataSet["Hora"] == 1200]

In [7]:
#Verificando dataframe para os dados disponibilizados às 00:00 h
dataSet00.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
0,1968-01-01,0,NaN,30.6,NaN,11.1,4.4,25.14,73.50,2.666667
2,1968-01-02,0,NaN,31.0,NaN,11.9,6.5,24.92,74.75,1.666667
4,1968-01-03,0,NaN,31.6,NaN,11.3,5.0,24.90,77.25,2.333333
6,1968-01-04,0,NaN,31.2,NaN,10.3,4.0,24.82,80.75,3.666667
8,1968-01-05,0,NaN,30.8,NaN,11.0,4.2,24.78,76.00,4.000000


In [8]:
#Verificando dataframe para os dados disponibilizados às 12:00 h
dataSet12.head()

,Data,Hora,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento
1,1968-01-01,1200,0.0,NaN,21.4,NaN,NaN,NaN,NaN,NaN
3,1968-01-02,1200,0.0,NaN,19.6,NaN,NaN,NaN,NaN,NaN
5,1968-01-03,1200,0.0,NaN,18.5,NaN,NaN,NaN,NaN,NaN
7,1968-01-04,1200,0.0,NaN,20.7,NaN,NaN,NaN,NaN,NaN
9,1968-01-05,1200,0.0,NaN,19.9,NaN,NaN,NaN,NaN,NaN


In [0]:
#Transformando a data no índice de cada dataframe
dataSet00 = dataSet00.set_index("Data")
dataSet12 = dataSet12.set_index("Data")

In [0]:
#Descartando a coluna "Hora"
dataSet00, dataSet12 = dataSet00.drop(columns = "Hora"), dataSet12.drop(columns = "Hora")

In [11]:
#Juntando os dados em um mesmo dataframe (no caso, o dataSet00)
for i in dataSet00.index:
    try:
        dataSet00["Prec"].loc[i] = dataSet12["Prec"].loc[i]
        dataSet00["TempMin"].loc[i] = dataSet12["TempMin"].loc[i]
    except:
        print("Data " + str(i.day) + "/" + str(i.month) + "/" + str(i.year) + " Não encontrada!")

Data 5/7/1986 Não encontrada!
Data 20/7/1986 Não encontrada!
Data 3/6/1989 Não encontrada!
Data 12/6/1989 Não encontrada!
Data 4/12/1992 Não encontrada!
Data 26/5/2018 Não encontrada!
Data 1/8/2018 Não encontrada!
Data 27/10/2018 Não encontrada!


In [12]:
#Atribuindo à variável dataSet o DataFrame atualizado
dataSet = dataSet00
#Verificando dimensões do DataFrame
print(dataSet.shape)

(13502, 8)


In [0]:
#Eliminando dias que estejam faltando variáveis
dataSet = dataSet.dropna()

In [14]:
#Verificando dimensões do DataFrame após excluir linhas com problema
print(dataSet.shape)

(11911, 8)


In [0]:
#Transformando a coluna Data de volta em uma coluna de informações do dataSet
dataSet = dataSet.reset_index()

# Considerando os seguintes intervalos: 
- Outono: 20 de março a 21 de junho (fechado em 20/03 e aberto em 21/06)
- Inverno: 21 de junho a 22 de setembro (fechado em 21/06 e aberto em 22/09)
- Primavera: de 22 de setembro a 21 de dezembro (fechado em 22/09 e aberto em 21/12)
- Verão: de 21 de dezembro a 20 de março (fechado em 21/12 e aberto em 20/03)

In [0]:
#Função que retorna a estação do ano para uma determinada data
def season(data):
    if (data.month > 3 and data.month < 6) or (data.month == 3 and data.day >= 20) or (data.month == 6 and data.day < 21):
        return "Out"
    if (data.month > 6 and data.month < 9) or (data.month == 6 and data.day >= 21) or (data.month == 9 and data.day < 22):
        return "Inv"
    if (data.month > 9 and data.month < 12) or (data.month == 9 and data.day >= 22) or (data.month == 12 and data.day < 21):
        return "Pri"
    else:
        return "Ver"

In [0]:
#Identificando a estação do ano na base de dados
estacao = []
for i in dataSet.index:
    estacao.insert(i, season(dataSet.loc[i, "Data"]))

In [18]:
#Contando quantos dados existem para cada estação
print("Outono: " + str(estacao.count("Out")))
print("Inverno: " + str(estacao.count("Inv")))
print("Primavera: " + str(estacao.count("Pri")))
print("Verão: " + str(estacao.count("Ver")))

Outono: 3048
Inverno: 2988
Primavera: 2946
Verão: 2929


In [0]:
#Adicionando a estação do ano na base de dados
dataSet["Season"] = estacao

In [20]:
#Eliminando a coluna "Data" e visualizando dados
dataSet = dataSet.drop(columns="Data")
dataSet.head()

,Prec,TempMax,TempMin,Insolacao,EvapoPiche,TempCompMedia,UmidRelatMedia,VelocVento,Season
0,0.0,30.6,21.4,11.1,4.4,25.14,73.50,2.666667,Ver
1,0.0,31.0,19.6,11.9,6.5,24.92,74.75,1.666667,Ver
2,0.0,31.6,18.5,11.3,5.0,24.90,77.25,2.333333,Ver
3,0.0,31.2,20.7,10.3,4.0,24.82,80.75,3.666667,Ver
4,0.0,30.8,19.9,11.0,4.2,24.78,76.00,4.000000,Ver


In [0]:
#Separando a variável que será prevista (Y) em função das demais variáveis (X) e convertendo-os em um ndarray
X = np.array(dataSet.drop(columns = "Season"))
Y = np.array(dataSet["Season"])

In [0]:
#Efetuando uma transformação robusta nos dados - menor sensibilidade aos outliers
sc = RobustScaler(quantile_range = (10.0,90.0))
X = sc.fit_transform(X)

In [0]:
#Definindo uma função para fazer o Cross Validation com a SVM
def trainSVM(X, Y, paramC, paramG):
  clf = SVC(C = paramC, kernel = 'rbf', random_state = 7, gamma = paramG)
  score = cross_val_score(clf, X, Y, cv=5)
  return score

In [24]:
#Este processo faz parte da busca pelos melhores parametros C e Gamma para a SVM.
#Declaracao de variaveis para ajudar na escolha dos parametros que retornem o melhor numero de acertos
C_range = np.logspace(start = -10, stop = 10, num = 20)
gamma_range = np.logspace(start = -10, stop = 10, num = 20)
acertos_maior = 0
acerto_local = 0
C_maior = C_range[0]
gamma_maior = gamma_range[0]
contador_passo = 0
print("O numero de iteracoes necessarias sera de: " + str(len(C_range)*len(gamma_range)))

O numero de iteracoes necessarias sera de: 400


In [0]:
#Treinando o algoritmo exaustivamente com os parâmetros criados
for i in C_range:
  for j in gamma_range:
    contador_passo += 1
    print("passo: " + str(contador_passo))
    #Treinando o algoritmo
    score = trainSVM(X, Y, i, j)
    #Numero de acertos locais
    acerto_local = score.mean()
    if acerto_local > acertos_maior:
      acertos_maior = acerto_local
      C_maior = i
      gamma_maior = j

passo: 1
passo: 2
passo: 3
passo: 4
passo: 5
passo: 6
passo: 7
passo: 8
passo: 9
passo: 10
passo: 11
passo: 12
passo: 13
passo: 14
passo: 15
passo: 16
passo: 17
passo: 18
passo: 19
passo: 20
passo: 21
passo: 22
passo: 23
passo: 24
passo: 25
passo: 26
passo: 27
passo: 28
passo: 29
passo: 30
passo: 31
passo: 32
passo: 33
passo: 34
passo: 35
passo: 36
passo: 37
passo: 38
passo: 39
passo: 40
passo: 41
passo: 42
passo: 43
passo: 44
passo: 45
passo: 46
passo: 47
passo: 48
passo: 49


In [0]:
#Mostrando os resultados obtidos:
print("A maior taxa de acerto foi de: " + str(acertos_maior))
print("Com o parametro C = " + str(C_maior) + ", e com o parâmetro Gamma = " + str(gamma_maior))